In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime as dt

from scipy.spatial import Delaunay
from scipy.stats import linregress
from scipy.optimize import curve_fit
from getdist import plots, MCSamples
from iminuit import Minuit
from iminuit.cost import LeastSquares

from qubic.lib.Qgps import GPSAntenna
import qubic.lib.Calibration.Qfiber as ft
 
%matplotlib inline 

In [ ]:
# If True, allow plots for debug
DEBUG = True

# Import data

In [ ]:
### Build GPS data file path
data_path = "calsource_orientation.dat"

### Define the distance between the two antennas
distance_between_antennas = 1.3 # Just to initialize GPSAntenna

gps_antenna = GPSAntenna(data_path, distance_between_antennas)

names = np.array(['North', 'East', 'Down', 'Roll', 'Yaw'])
print(gps_antenna._datetime[8100])

del data_path, distance_between_antennas 

In [ ]:
index_ini = gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([dt.datetime(year=2025, month=3, day=21, hour=14, minute=20, second=26)]))[0]
print('Initial index = ', index_ini)
print('size observation times : ', len(gps_antenna._datetime))
index_stop = -1
print('Date when observation stoped : ', gps_antenna._datetime[index_stop])

In [ ]:
gps_antenna.plot_gps_data(index_ini, index_stop)
gps_antenna.plot_gps_data_plotly(index_ini, index_stop)

# Build observation time indices

In [ ]:
# Array containing the starting time of each configuration
import array


array_hours_begins = np.array([14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18])
array_minutes_begins = np.array([20, 26, 31, 38, 44, 49, 55, 1, 7, 13, 18, 24, 29, 35, 41, 48, 52, 0, 3, 10, 28, 33, 39, 45, 51, 56, 3, 8, 15, 21, 26, 32, 38, 44, 49, 0, 6, 12, 17, 23, 29])
array_seconds_begins = np.array([33, 10, 50, 25, 0, 32, 25, 35, 30, 20, 45, 5, 23, 3, 5, 3, 55, 0, 55, 0, 30 ,50, 40, 30, 0, 50, 10, 40, 55, 30, 55, 40, 5, 0, 20, 55, 30, 5, 35, 40, 5])

if array_hours_begins.size != array_minutes_begins.size or array_hours_begins.size != array_seconds_begins.size:
    print(array_hours_begins.size, array_minutes_begins.size, array_seconds_begins.size)
    raise ValueError("The arrays must have the same size")

# Array containing the seconds of each configuration
array_seconds_ending = np.zeros(array_hours_begins.shape, dtype=int)

# Build the array of datetime associated with each configuration, to delimit the time when the antenna are moving or not
#! Each configuration last at least 5 minutes
array_datetime_begin = np.array([])
array_datetime_end = np.array([])
for index in range(array_hours_begins.shape[0]):
    array_datetime_begin = np.append(array_datetime_begin, dt.datetime(year=2025, month=3, day=21, hour=array_hours_begins[index], minute=array_minutes_begins[index], second=array_seconds_begins[index]))
    if array_minutes_begins[index] > 54:
        # Test to avoid minutes > 60
        print(array_minutes_begins[index])
        array_datetime_end = np.append(array_datetime_end, dt.datetime(year=2025, month=3, day=21, hour=array_hours_begins[index] + 1, minute=array_minutes_begins[index] + 5 - 60, second=array_seconds_ending[index]))
    else:
        array_datetime_end = np.append(array_datetime_end, dt.datetime(year=2025, month=3, day=21, hour=array_hours_begins[index], minute=array_minutes_begins[index] + 5, second=array_seconds_ending[index]))
array_datetime_end[34] = dt.datetime(year=2025, month=3, day=21, hour=17, minute=53, second=29)
array_datetime_begin[35] = dt.datetime(year=2025, month=3, day=21, hour=18, minute=2, second=22)
array_datetime_end[36] = dt.datetime(year=2025, month=3, day=21, hour=18, minute=11, second=45)

index_array_begin = np.array([], dtype=int)
index_array_end = np.array([], dtype=int)
for index in range(array_datetime_begin.size):
    index_array_begin = np.append(index_array_begin, gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([array_datetime_begin[index]])))
    index_array_end = np.append(index_array_end, gps_antenna.get_observation_indices(gps_antenna._datetime, np.array([array_datetime_end[index]])))
        
del array_hours_begins, array_minutes_begins, array_seconds_begins, array_seconds_ending, array_datetime_begin, array_datetime_end, index

## Test

In [ ]:
# if DEBUG:
#     fig = go.Figure()
#     for index in range(int(index_array_begin.size)):
        
#         fig.add_trace(go.Scatter(x=gps_antenna._datetime[index_ini:], y=gps_antenna.roll[index_ini:]))
#         fig.add_vline(x=gps_antenna._datetime[index_array_begin[index]], line_dash="dash", line_color="red")
#         fig.add_vline(x=gps_antenna._datetime[index_array_end[index]], line_dash="dash", line_color="blue")
#         fig.update_layout(
#             title='GPS Roll Angle',
#             xaxis_title='Date',
#             yaxis_title='Angle (rad)',
#             yaxis_range=[0, 2*np.pi]
#         )
    
#     fig.show()
#     del index

In [ ]:
# Plot to verify the previous delimitation
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpN[index_ini:], color = color_r, label = 'North component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpE[index_ini:], color = color_b, label = 'East component')
ax1.plot(gps_antenna._datetime[index_ini:], gps_antenna.rpD[index_ini:], color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.roll[index_ini:], color = color_a, label = 'Roll angle')
ax2.plot(gps_antenna._datetime[index_ini:], gps_antenna.yaw[index_ini:], color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

ax2.vlines(gps_antenna._datetime[index_array_begin], -1, 5, 'r', linestyles='--')
ax2.vlines(gps_antenna._datetime[index_array_end], -1, 5, 'b', linestyles='--')

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c,

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=gps_antenna._datetime[index_ini:index_stop], y=gps_antenna.rpN[index_ini:],
               name="North component", line=dict(color='red')),secondary_y=False)

fig.add_trace(
    go.Scatter(x=gps_antenna._datetime[index_ini:index_stop], y=gps_antenna.rpE[index_ini:],
               name="East component", line=dict(color='blue')),secondary_y=False)

fig.add_trace(
    go.Scatter(x=gps_antenna._datetime[index_ini:index_stop], y=gps_antenna.rpD[index_ini:],
               name="Up component", line=dict(color='green')), secondary_y=False)

# Add angles on secondary y-axis
fig.add_trace(
    go.Scatter(x=gps_antenna._datetime[index_ini:index_stop], y=gps_antenna.roll[index_ini:],
               name="Roll angle", line=dict(color='pink')),secondary_y=True,)

fig.add_trace(
    go.Scatter(x=gps_antenna._datetime[index_ini:index_stop], y=gps_antenna.yaw[index_ini:],
               name="Yaw angle", line=dict(color='brown')),secondary_y=True,)

# Add vertical lines
for i in range(len(index_array_begin)):
    fig.add_vline(
        x=gps_antenna._datetime[index_array_begin[i]],
        line_width=1,
        line_dash="dash",
        line_color="red",
        layer='above')
    
    fig.add_vline(
        x=gps_antenna._datetime[index_array_end[i]],
        line_width=1,
        line_dash="dash",
        line_color="blue",
        layer='above')

# Update layout
fig.update_layout(
    title="Position vector components",
    xaxis_title="Date",
    legend=dict(x=1.1, y=1),
    width=1600,
    height=800,)

fig.update_yaxes(title_text="Position Vector Components (m)", secondary_y=False, color="red")
fig.update_yaxes(title_text="Angles (rad)", secondary_y=True, color="pink")

fig.show()

In [ ]:
# Function to remove the movement between each configurations
def only_data(array, indices_begin, indices_end):
    data_array = np.array([])
    
    if indices_begin.size != indices_end.size:
        raise ValueError("The number of begin and end indices must be the same.")
    
    for i in range(int(indices_begin.size)):
        data_array = np.append(data_array, array[indices_begin[i]:indices_end[i]])
    
    return data_array

In [ ]:
data_rpN = only_data(gps_antenna.rpN, index_array_begin, index_array_end)
data_rpE = only_data(gps_antenna.rpE, index_array_begin, index_array_end)
data_rpD = only_data(gps_antenna.rpD, index_array_begin, index_array_end)
data_roll = only_data(gps_antenna.roll, index_array_begin, index_array_end)
data_yaw = only_data(gps_antenna.yaw, index_array_begin, index_array_end)
data_datetime = only_data(gps_antenna._datetime, index_array_begin, index_array_end)

In [ ]:
# Plot to verify the filtering of the time when the antennas are moving
fig, ax1 = plt.subplots(figsize = (15,5))

color_a = 'tab:pink'
color_r = 'tab:red'
color_b = 'tab:blue'
color_d = 'tab:green'
color_c = 'tab:brown'

ax1.set_xlabel('Date')
ax1.set_ylabel('Position Vector Components (m)', color = color_r)
ax1.plot(data_rpN, color = color_r, label = 'North component')
ax1.plot(data_rpE, color = color_b, label = 'East component')
ax1.plot(data_rpD, color = color_d, label = 'Up component')

ax2 = ax1.twinx()

ax2.plot(data_roll, color = color_a, label = 'Roll angle')
ax2.plot(data_yaw, color = color_c, label = 'Yaw angle')
ax2.set_xlabel('Date')
ax2.set_ylabel('Angles (rad)', color = color_a)

fig.tight_layout()
ax1.set_title("Position vector components")
fig.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

del fig, ax1, ax2, color_a, color_r, color_b, color_d, color_c